In [122]:
import mysql.connector as connector
from mysql.connector import errorcode, errors, pooling
import datetime as dt

In [123]:
dbconfig = {
    "database": "little_lemon",
    "user": "root",
    "password": "mysql_pass",
    "host": "localhost",
}

try:
    pool_name = "pool_b"
    pool_size = 2
    cnxpool = connector.pooling.MySQLConnectionPool(
        pool_name = pool_name,
        pool_size = pool_size,
        **dbconfig
    )
    print(f"The connection pool is created with a name: {pool_name}")
    print(f"The pool size is: {pool_size}")

except connector.Error as err:
    print(f"Error code: {err.errno}")
    print(f"Error message {err.msg}")

The connection pool is created with a name: pool_b
The pool size is: 2


In [124]:
guests = ["Anees", "Bald", "Jay"]
guests_connections = {}

for guest in guests:
    try:
        connection = cnxpool.get_connection()
        guests_connections[guest] = connection
        print(f"[{guest}] is connected.\n")
        
    except connector.PoolError:
        print("No more connections are available.")
        print("Adding new connection in the pool.")
        cnxpool.add_connection()
        print("A new connection is added in the pool.\n")
        connection = cnxpool.get_connection()
        guests_connections[guest] = connection
        print(f"[{guest}] is connected.\n")

[Anees] is connected.

[Bald] is connected.

No more connections are available.
Adding new connection in the pool.
A new connection is added in the pool.

[Jay] is connected.



In [125]:
cursor = connection.cursor()

In [126]:
insert_bookings = """
INSERT INTO bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES (8, 'Anees', 'Java', '18:00:00', 6),
(5, 'Bald', 'Vin', '19:00:00', 6),
(12, 'Jay', 'Kon', '19:30:00', 6);
"""

cursor.execute(insert_bookings)
connection.commit()

## Stored Procedures

In [127]:
create_peak_hour = """
CREATE PROCEDURE PeakHours ()
BEGIN 
SELECT HOUR(BookingSlot) AS hour_time, COUNT(BookingID) as number_of_bookings
FROM Bookings
GROUP BY hour_time
ORDER BY number_of_bookings DESC;
END;
"""

cursor.execute(create_peak_hour)

ProgrammingError: 1304 (42000): PROCEDURE PeakHours already exists

In [128]:
cursor.callproc('PeakHours')

()

In [129]:
results = next(cursor.stored_results())

print(results.column_names)
for row in results.fetchall():
    print(row)

('hour_time', 'number_of_bookings')
(19, 6)
(18, 3)
(15, 1)
(17, 1)
(20, 1)


In [130]:
create_guest_status = """
CREATE PROCEDURE GuestStatus()
BEGIN
SELECT CONCAT(GuestFirstName, " ", GuestLastName) as name, 
CASE
	WHEN e.Role = "Manager" THEN "Ready to pay"
    WHEN e.Role = "Assistant Manager" THEN "Ready to pay"
	WHEN e.Role = "Head Chef" THEN "Ready to serve"
    WHEN e.Role = "Assistant Chef" THEN "Preparing Order"
    WHEN e.Role = "Head Waiter" THEN "Order served"
END AS status
FROM bookings b
LEFT JOIN employees e on (b.employeeID = e.employeeID);
END;
"""

cursor.execute(create_guest_status)

ProgrammingError: 1304 (42000): PROCEDURE GuestStatus already exists

In [131]:
cursor.callproc('GuestStatus')

()

In [132]:
results = next(cursor.stored_results())

print(results.column_names)
for row in results.fetchall():
    print(row)

('name', 'status')
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')
('Anees Java', None)
('Bald Vin', None)
('Jay Kon', None)
('Anees Java', None)
('Bald Vin', None)
('Jay Kon', None)


In [133]:
create_sales_report = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
SELECT SUM(BillAmount), ROUND(AVG(BillAmount), 2), MAX(BillAmount), MIN(BillAmount)
FROM orders;
END
"""

cursor.execute(create_sales_report)

ProgrammingError: 1304 (42000): PROCEDURE BasicSalesReport already exists

In [134]:
cursor.callproc('BasicSalesReport')

()

In [135]:
results = next(cursor.stored_results())

print(results.column_names)
for row in results.fetchall():
    print(row)

('SUM(BillAmount)', 'ROUND(AVG(BillAmount), 2)', 'MAX(BillAmount)', 'MIN(BillAmount)')
(Decimal('243'), Decimal('48.60'), 86, 37)


## Short Report

In [136]:
cursor.execute("SELECT Name, EmployeeID FROM employees where Role = 'Manager'")

In [137]:
print(cursor.column_names)
cursor.fetchall()

('Name', 'EmployeeID')


[('Mario Gollini', 1)]

In [138]:
cursor.execute("SELECT Name, Role FROM employees WHERE Annual_Salary = (SELECT MAX(Annual_Salary) FROM employees)")

In [139]:
cursor.fetchall()

[('Mario Gollini', 'Manager')]

In [140]:
cursor.execute("SELECT COUNT(BookingID) FROM bookings WHERE BookingSlot BETWEEN '18:00' AND '20:00'")

In [141]:
cursor.fetchall()

[(10,)]

In [142]:
reception_guest_query = """
SELECT CONCAT(GuestFirstName, " ", GuestLastName) as full_name, BookingID 
FROM bookings 
WHERE EmployeeID = ALL (
SELECT EmployeeID 
FROM employees 
WHERE Role = 'Receptionist'
)
ORDER BY BookingSlot
"""

In [143]:
cursor.execute(reception_guest_query)

In [144]:
print(cursor.column_names)
for row in cursor.fetchall():
    print(row)

('full_name', 'BookingID')
('Anees Java', 7)
('Anees Java', 10)
('Bald Vin', 8)
('Bald Vin', 11)
('Jay Kon', 9)
('Jay Kon', 12)


## Pool Connection

In [145]:
buffered_cursor = guests_connections['Anees'].cursor(buffered = True)

In [146]:
get_kitchen_order = """
SELECT BookingSlot, CONCAT(GuestFirstName, " ", GuestLastName) as guest_name, e.Name as employee_name, e.Role
FROM bookings b
INNER JOIN employees e on (b.EmployeeID = e.EmployeeID)
WHERE e.Role LIKE "%Chef"
ORDER BY BookingSlot
"""

buffered_cursor.execute(get_kitchen_order)

In [147]:
for row in buffered_cursor.fetchall():
    print(f"BookingSlot: {row[0]}")
    print(f"Guest_name: {row[1]}")
    print(f"Assigned to: {row[2]} [{row[3]}]\n")


BookingSlot: 15:00:00
Guest_name: Vanessa McCarthy
Assigned to: Giorgos Dioudis [Head Chef]

BookingSlot: 17:30:00
Guest_name: Marcos Romero
Assigned to: Fatma Kaya [Assistant Chef]

